<a href="https://colab.research.google.com/github/kerolos98/resumeparserlib/blob/main/headers_exctractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf
!pip install transformers
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 27.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_lg: filename=en_core_web_lg-2.3.1-py3-none-any.whl size=782936122 sha256=6f9a09768724e96dacb24e33a43b97c040723c24c9355ec70df2d41eecd0605d
  Stored in directory: /root/.cache/pip/wheels/8b/bb/bb/bdc918f4b37d930a1be9ed876e7b2c2ee518a34803d78a248e
Successfully built en_core_web_lg
  Attempting uninstall: en_core_web_lg
    Found existing installation: en-core-web-lg 3.4.1
    Uninstalling en-core-web-lg-3.4.1:
      Successfully uninstalled en-core-web-lg-3.4.1
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# **outputs of the same pdf**


*   /content/drive/MyDrive/مجلد بلا عنوان/Resume/3341114f-ff74-4ccc-82be-61adf99598b3bhanu_sai-converted.pdf



In [94]:
import sys
import subprocess
import re


def convert_to(folder, source, timeout=None):
    args = [libreoffice_exec(), '--headless', '--convert-to', '.pdf', '--outdir', folder, source]

    process = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=timeout)
    filename = re.search('-> (.*?) using filter', process.stdout.decode())

    return filename.group(1)


def libreoffice_exec():
    # TODO: Provide support for more platforms
    if sys.platform == 'darwin':
        return '/Applications/LibreOffice.app/Contents/MacOS/soffice'
    return 'libreoffice'

In [95]:

import fitz
import spacy 
nlp = spacy.load('en_core_web_lg')
def scrape(filePath):
    results = [] # list of tuples that store the information as (text, font size, font name)
    size = []
    fonts=[]
    combination=[]
    pdf = fitz.open(filePath) # filePath is a string that contains the path to the pdf
    for page in pdf:
        dict = page.get_text("dict")
        blocks = dict["blocks"]
        for block in blocks:
            if "lines" in block.keys():
                spans = block['lines']
                for span in spans:
                    data = span['spans']
                    for lines in data:
                      if len(lines['text'])>=2: 
                            results.append([lines['text'], lines['size'], lines['font']])
                            size.append(lines['size'])
                            fonts.append(lines['font'])
                            combination.append((lines['size'], lines['font']))

                            # lines['text'] -> string, lines['size'] -> font size, lines['font'] -> font name
    pdf.close()
    return results ,size,fonts,combination


In [111]:
data,size,fonts,combination=scrape('/content/drive/MyDrive/kerolos emad__ (3)-1.pdf')




In [112]:
def bold_data(data):
    data_bold=[]
    for d in data:
      if 'Bold' in d[2] or d[0].isupper() :
        if d[0]!=' ':
          data_bold.append(d)
    return data_bold      

In [134]:
def bold_filter(data):
  keywords=['summary','work experience','phone number','contact','email','address','education','Languages','projects','skills','ACHIEVEMENTS','Details','Links','Hobbies']
  new=[]
  for d in data:
    for key in keywords:
      search = nlp(d[0].lower())
      main = nlp(key)
      if main.similarity(search)>0.5:
        if d in new:
          continue
        new.append(d)
  return new 

In [135]:
headers=bold_filter(bold_data(data))

<ipython-input-134-19fe8a2c0477>:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if main.similarity(search)>0.5:


In [136]:
headers

[['LINKEDIN:', 9.0, 'CenturyGothic'],
 ['EMAIL:', 9.0, 'CenturyGothic'],
 ['EDUCATION', 11.0, 'CenturyGothic-Bold'],
 ['Graduation Project:', 9.0, 'CenturyGothic-Bold'],
 ['Coursework:', 9.0, 'CenturyGothic-Bold'],
 ['WORK EXPERIENCE', 11.0, 'CenturyGothic-Bold'],
 ['Predictive Analytics Modeler Training – IBM', 9.0, 'CenturyGothic-Bold'],
 ['Projects:', 9.0, 'CenturyGothic-Bold'],
 ['FEEDBACKS', 7.699999809265137, 'CenturyGothic'],
 ['REVIEW MODEL', 7.699999809265137, 'CenturyGothic'],
 [' SKILLS', 11.0, 'CenturyGothic-Bold']]

In [137]:
from numpy import True_
import re
def data_organizer(data,headers):
    dict_data=dict()
    for i in range(len(headers)):
      try:
          dict_data[headers[i][0]]=data[data.index(headers[i])+1:data.index(headers[i+1])]
      except IndexError:  
          dict_data[headers[i][0]]=data[data.index(headers[i])+1:-1]
    return dict_data
def size_structures(data,state):
    temp=0
    memory=''
    tree=dict()
    for d in data:
        sentence = re.sub(r"\s+", "", d[0], flags=re.UNICODE)
        if sentence!='':
            if d[1]>=temp:
              if state==True:
                if 'Bold' in d[2]:
                  memory=d[0]
                  temp=d[1]
                  tree[d[0]]=[]
                  
              else:  
                  memory=d[0]
                  temp=d[1]
                  tree[d[0]]=[]
            else:
              tree[memory].append(d[0]) 
    return tree
def bold_state(data):
  state=False
  for d in data :
    if 'Bold' in d[2]:
      state=True
      break
  return state       

In [149]:
dict_data=data_organizer(data,headers)

In [150]:
dict_data.keys()

dict_keys(['LINKEDIN:', 'EMAIL:', 'EDUCATION', 'Graduation Project:', 'Coursework:', 'WORK EXPERIENCE', 'Predictive Analytics Modeler Training – IBM', 'Projects:', 'FEEDBACKS', 'REVIEW MODEL', ' SKILLS'])

In [151]:
dict_data['EDUCATION']

[['BSc in Computer Science - Higher Technological Institute (HTI) 10',
  9.0,
  'CenturyGothic-Bold'],
 ['th', 5.399999618530273, 'CenturyGothic-Bold'],
 ['of', 9.0, 'CenturyGothic-Bold'],
 ['Ramadan City', 9.0, 'CenturyGothic-Bold'],
 ['Attended: 2016 - 2020', 9.0, 'CenturyGothic'],
 ['GPA: 3.2, Graduated(HTI-CS-2020)', 9.0, 'CenturyGothic']]

In [152]:
#section content size seperation
def size_seperator(dict_data):
  for k,v in dict_data.items():
      dict_data[k]=size_structures(v,bold_state(v))
  return  dict_data   

In [153]:
dict_data=size_seperator(dict_data)

In [157]:
dict_data['EMAIL:']

{'kemad57@gmail.com': []}

In [144]:
from textblob import TextBlob

def date_period(input_string):
      # a generator will be returned by the datefinder module. I'm typecasting it to a list. Please read the note of caution provided at the bottom.
      matches = list(datefinder.find_dates(input_string))

      if len(matches) > 0:
          # date returned will be a datetime.datetime object. here we are only using the first match.
          return matches
      else:
          return ('No dates found')
def discription(ents,text):
  for k,v in ents.items():
      text=text.replace(v,' ')
  return text
def entities(text):
    doc = nlp(text)
    counter=0
    entities=dict()
    for entity in doc.ents:
      entities[entity.label_+str(counter)]= entity.text
      counter+=1
    return   entities

In [43]:
def dic_check(dict0):
  state=False
  for k,v in dict0.items():
    if len(v)>0:
      state=True
      break
  return  state   

In [64]:
def _correct(text):
        
      gfg = TextBlob(text)
      
      # using TextBlob.correct() method
      gfg = gfg.correct()
      return str(gfg)

In [74]:
def structured_entities_based(dict_data):
    title=[]
    dates=[]
    jsons=dict()
    for k,v in dict_data.items():
          loader=dict()
          state=dic_check(v)
          if state==True:
            
            for c,d in v.items():
              obj_data=dict()
              text=''.join(d)
              ents1=entities(c)
              ents2=entities(text)
              discriptions='no data'
              if  bool(ents2) :
                  discriptions=_correct(discription(ents2,str(text)))
              obj_data['title']=c
              obj_data['full_text']=text
              obj_data['entities']=[ents1,ents2]
              obj_data['discription']=discriptions
              loader[c]=obj_data
              
            jsons[k]=loader
            
          else:
            obj_data=dict()
            text=''.join(v.keys())
            ents=entities(text)
            discriptions='no data'
            if  bool(ents) :
                  discriptions=_correct(discription(ents,str(text)))
            obj_data['full_text']=text
            obj_data['entities']=ents
            obj_data['discription']=discriptions
            jsons[k]=obj_data
    return  jsons      
 



In [89]:
dict_data

{'Phone ': {'full_text': ':  +91 7207999519',
  'entities': {},
  'discription': 'no data'},
 'Notice Period : Immediate Joiner': {'full_text': '',
  'entities': {},
  'discription': 'no data'},
 'Email   :  akbarbasha5943@gmail.com': {'full_text': '',
  'entities': {},
  'discription': 'no data'},
 'Professional Summary:': {'full_text': '3.4 Years of IT experience in  ',
  'entities': {'DATE0': '3.4 Years'},
  'discription': '  of of experience in  '},
 'Manual Testing  ': {'full_text': ' API Testing  POSTMAN ',
  'entities': {},
  'discription': 'no data'},
 'understanding': {'full_text': ' the ',
  'entities': {},
  'discription': 'no data'},
 'Requirements': {'full_text': ' and ',
  'entities': {},
  'discription': 'no data'},
 'analysis': {'full_text': ' the ', 'entities': {}, 'discription': 'no data'},
 'testing scope': {'full_text': 'items',
  'entities': {},
  'discription': 'no data'},
 'Tracking': {'full_text': ' the ', 'entities': {}, 'discription': 'no data'},
 'changes of 

In [160]:
import json
def file_extraction(path):
  try:
    data,size,fonts,combination=scrape(path)
  except:
    result=convert_to('TEMP Directory',  path, timeout=30)
    data,size,fonts,combination=scrape(result)
  headers=bold_filter(bold_data(data))
  dict_data=  data_organizer(data,headers)
  #json file based on text featrures
  json_text=dict_data
  with open("json_text.json","w") as f:
    json.dump(json_text,f)
  # json files with more structure based on size and font style 
  dict_data=size_seperator(dict_data)
  json_size_font=dict_data
  with open("json_size_font.json","w") as f:
    json.dump(json_size_font,f)
  #json file based on extracting section entities and descriptions
  dict_data= structured_entities_based(dict_data)
  with open("entities_data.json","w") as f:
    json.dump(dict_data,f)


In [161]:
file_extraction('/content/drive/MyDrive/Stockholm-Resume-Template-Simple (2).pdf')

<ipython-input-134-19fe8a2c0477>:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if main.similarity(search)>0.5:


In [88]:
!pip freeze > requirements.txt 